# Introduction 

**LangChain** is an opensource framework for connecting LLMs and datasources under a unified syntax. 

It allows the creation of scalable modular aplications.

In [ ]:
import os

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(repo_id='tiiuae/falcon-7b-instruct', huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY'))

# Predict the words following the text in question
question = 'Whatever you do, take care of your shoes'
output = llm.invoke(question)

print(output)

In [ ]:
from langchain_openai import OpenAI

lm = OpenAI(model="gpt-3.5-turbo-instruct", api_key=os.getenv('OPENAI_API_KEY'))

# Predict the words following the text in question
question = 'Whatever you do, take care of your shoes'
output = llm.invoke(question)

print(output)

# Prompt Templates

In [ ]:
from langchain_core.prompts import PromptTemplate

template = 'You are an artificial intelligence assistant, answer the question. {question}'
prompt_template = PromptTemplate(template=template, input_variables=['question'])

print(prompt_template.invoke({'question': 'What is LangChain?'}))


In [ ]:
from langchain_core.prompts import ChatPromptTemplate 

prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are soto zen master Roshi.'), 
        ('human', 'What is the essence of Zen?'),
        ('ai', 'When you are hungry, eat. When you are tired, sleep.'),
        ('human', 'Respond to the question: {question}'),   
    ]
)

In [ ]:
from langchain_openai import ChatOpenAI 

llm = ChatOpenAI(model = 'gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))

llm_chain = prompt_template | llm 
question = 'what is the sound of one hand clapping?' 
response = llm_chain.invoke({'question': question})
print(response.content)

In [ ]:
# Create a prompt template from the template string
template = "You are an artificial intelligence assistant, answer the question. {question}"
prompt = PromptTemplate(template=template, input_variables=['question'])

# Create a chain to integrate the prompt template and LLM
llm = HuggingFaceEndpoint(repo_id='tiiuae/falcon-7b-instruct', huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY'))
llm_chain = prompt | llm 

question = "How does LangChain make LLM application development easier?"
print(llm_chain.invoke({"question": question}))

# Managing chat model memory 

Allows: 
- follow up questions
- response iteration and expansion
- personalization

Limited by the model context window (amount of input text a model can consider at once when generating a response)

Langchain has 3 classes to model this context window: 
- ChatMessageHistory: full exchange of messages.
- ConversationBufferMemory: keeps a certain number amount of messages 
- ConversationSummaryMemory: keeps a summary of the conversation.



In [ ]:
from langchain.memory import ChatMessageHistory 
from langchain_openai import ChatOpenAI 

llm = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))

history = ChatMessageHistory() 
history.add_ai_message('Hi! Ask me anything about langchain.')
history.add_user_message('Describe a metaphor for learning LangChain in one sentence.')

response = llm.invoke(history.messages)

print(response.content)

In [ ]:
history.add_user_message('Sumarize the preceding sentence in fewer words')
response = llm.invoke(history.messages) 
print(response.content)

In [ ]:
from langchain.memory import ConversationBufferMemory 
from langchain.chains import ConversationChain 

llm = ChatOpenAI(model = 'gpt-40-mini', api_key = os.getenv('OPENAI_API_KEY'))

memory = ConversationBufferMemory(size=4) 
buffer_chain = ConversationChain(llm=llm, memory=memory)


In [ ]:
from langchain.memory import ConversationSummaryMemory

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.getenv('OPENAI_API_KEY'))

# Define a summary memory that uses an OpenAI chat model
memory = ConversationSummaryMemory(llm=llm)

# Define the chain for integrating the memory with the model
summary_chain = ConversationChain(llm=llm, memory=memory, verbose=True)

# Invoke the chain with the inputs provided
summary_chain.invoke("Describe the relationship of the human mind with the keyboard when taking a great online class.")
summary_chain.invoke("Use an analogy to describe it.")

# Sequential Chains

In sequential chains the output of one chain becomes the input of the next.

In [ ]:
# Create a prompt template that takes an input activity
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

# Create a prompt template that places a time constraint on the output
time_prompt = PromptTemplate(
    input_variables=['learning_plan'],
    template="I only have one week. Can you create a plan to help me hit this goal: {learning_plan}."
)

# Invoke the learning_prompt with an activity
print(learning_prompt.invoke({"activity": "Swimming"}))

In [ ]:
from langchain_core.output_parsers.string import StrOutputParser

learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

time_prompt = PromptTemplate(
    input_variables=["learning_plan"],
    template="I only have one week. Can you create a plan to help me hit this goal: {learning_plan}."
)

# Complete the sequential chain with LCEL
seq_chain = ({"learning_plan": learning_prompt | llm | StrOutputParser()}
    | time_prompt
    | llm
    | StrOutputParser())

# Call the chain
print(seq_chain.invoke({"activity": "fishing"}))

# Agents 

Agents use LLMs to take actions 

Tools are functions called by the agent 

ReAct (Reasoning and Acting) agents are a type of agents

LangGraph is a branch of langChain centered around agents systems, decoupling our implementation from any specific vendor.



In [ ]:
from langgraph.prebuilt import create_react_agent 
from langchain.agents import load_tools 

llm = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
tools = load_tools(['llm-math'], llm=llm)
agent =create_react_agent(llm, tools)

messages = agent.invoke({'messages': [('human', 'What is the square root of 101?')]})
print(messages)

# Creating tools

In [ ]:
from langchain.agents import load_tools
tools = load_tools(['llm-math'], llm=llm) 
print(tools[0].name)
print(tools[0].description) #The llm will invoke this tool depending on its description 
print(tools[0].return_direct) # Indicates if the LLM should stop after invoking this tool 



In [ ]:


def financial_report(company_name:str, revenue:int, expenses:int) -> str:
    '''Generate a financial report for a company that calculates net income.''' 
    
    net_income = revenue - expenses 
    
    report = f'Financial report for {company_name}:\n'
    report += f'Revenue: ${revenue}\n'
    report += f'expenses: ${expenses}\n'
    report += f'Net income: ${net_income}\n'
    
    return report

print(financial_report('Totos motos', 3245345, 345345))

Lets create a tool out of that function: 

In [ ]:
from langchain_core.tools import tool 

@tool 
def financial_report(company_name:str, revenue:int, expenses:int) -> str:
    '''Generate a financial report for a company that calculates net income.''' 
    
    net_income = revenue - expenses 
    
    report = f'Financial report for {company_name}:\n'
    report += f'Revenue: ${revenue}\n'
    report += f'expenses: ${expenses}\n'
    report += f'Net income: ${net_income}\n'
    
    return report


print(financial_report.name)
print(financial_report.description) #The llm will invoke this tool depending on its description 
print(financial_report.return_direct) # Indicates if the LLM should stop after invoking this financial_report
print(financial_report.args)

# Retrieval Augmented Generation (RAG) 

Pretrined LLM dont have access to external data outside of the training one.

RAG is a technique to allow the LLM to use external data. It makes use of embeddings to retrieve relevant information to integrate into the prompt. 

In langchain, there are 3 steps to use RAG: 
- Document Loader: langchain provides document loaders for common file types like pdf, csv... 3rd parties allow for other file types loaders.
- Splitting (information into chunks): to avoid missing context, there is often a bit of chunk overlap. The splitting strategy as well as the chunck size will depend on the problem at hand.
- Storage + Retrieval

In [ ]:
from langchain_community.document_loaders import PyPDFLoader 
loader = PyPDFLoader("/Users/el_fer/Downloads/Confidences d’un petit cochon (Pierrette Dubé) (Z-Library).pdf")

data=loader.load() 
print(data[0])

In [ ]:
from langchain_text_splitters import CharacterTextSplitter 

quote = 'one machine can do the work of fifty ordinary humans. no machine can do the work of one extraordinary human'
ct_splitter = CharacterTextSplitter(
    separator = '.', 
    chunk_size = 20, 
    chunk_overlap=4
)

docs =ct_splitter.split_text(quote)
print(docs)

The problem of this method is that it uses present separators to split the text. 

Lets try to find a more robust splitting technique 

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter 

rc_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=24, 
    chunk_overlap=3
)

docs = rc_splitter.split_text(quote)
print(docs)

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma 

# PENDING: create the documents to be stored

embedding_function = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'), model='text-embedding-3-small')
vectorstore = Chroma.from_documents(docs, 
                                   embedding=embedding_function, 
                                   persist_directory='./chromaDB')

retriever = vectorstore.a_retriever(
    search_type='similarity', 
    search_kwargs={'k': 2}
)


In [ ]:
message = '''
Review and fix the following TechStack marketing copy with the following guidelines in consideration: 
Guidelines: 
{guidelines} 

Copy: 
{copy} 

Fixed Copy:
'''

prompt_template = ChatPromptTemplate.from_messages(['human', message])

In [ ]:
from langchain_core.runnables import RunnablePassthrough 

rag_chain = ({'guidelines': retriever, 'copy': RunnablePassthrough()}
             | prompt_template 
             | llm) 

